In [ ]:
#Import Modules and Function
import tweepy
import pymongo
import json
import pandas as pd
import time

In [ ]:
# TWITTER API Instance

# REQUEST TOKEN KEY (get keys/screts from API Account)
ACCESS_TOKEN="XXX" 
ACCESS_TOKEN_SECRET="XXX"
CONSUMER_KEY="XXX"
CONSUMER_SECRET="XXX"

# Twitter Authentication 
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

# create API instance
api = tweepy.API(auth)

In [ ]:
# MONGODB Instance

# Localhost
mongo_client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
mongo_db = mongo_client['datafyp']
mongo_collection = mongo_db['fyptwitter']

# NWX: MongoDB must be installed and deployed beforehand. 
# Here 127.0.0.1 means local server

In [ ]:
# Function to stream Tweets:
# --- Created_at, id, username, language, quote (full text), retweet (full text)

class myStreamListener(tweepy.StreamListener):
    """
    Derived Class for Stream-Listener
    """
    def __init__(self, mongoCollection, starttime='0600', endtime='1000'):
        # original init to be retained (init api)
        # adding mongoDB collection to class
        # add default start and end time
        super().__init__()
        self.mongoCollection = mongoCollection 
        self.starttime = starttime
        self.endtime = endtime

    # Child-Function to process Tweet-Statuses from Listener
    # write data into MongoDB directly
    def on_status(self, status):
        
        try:
            # crawl between specific timestamp
            if int(time.strftime("%H%M", time.localtime())) > int(self.starttime) and int(time.strftime("%H%M", time.localtime())) < int(self.endtime):
                # Get Quote-Tweet Full-Text, if available
                varQuote = True if ((status.is_quote_status == True) if hasattr(status, 'is_quote_status') else False) else False
                if varQuote:
                    varQuoteText = status.quoted_status.text if (status.quoted_status.truncated == False) else status.quoted_status.extended_tweet['full_text']
                else:
                    varQuoteText = ""
                varRetweet = True if ((status.retweeted_status) if hasattr(status, 'retweeted_status') else False) else False
                varFullText = status.text if (status.truncated == False) else status.extended_tweet['full_text']
                if varRetweet:
                    varFullText = status.retweeted_status.text if (status.retweeted_status.truncated == False) else status.retweeted_status.extended_tweet['full_text']

                self.mongoCollection.insert_one({
                    "created_at": status.created_at, 
                    "id": status.id,
                    "screen_name:": status.user.screen_name,
                    "lang": status.lang,
                    "is_quote_status": varQuote,
                    "is_retweet_status": varRetweet,
                    "full_text": varFullText,
                    "quote_text": varQuoteText
                })
        except:
            print('Error:',time.strftime("%Y/%m/%d-%H:%M:%S", time.localtime()))
            pass

    # Child-Function for Error 
    # Stop only when getting status 420 (rate limited), else continue
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
    def tryout(self):
        try:
            print(self.mongoCollection.find_one())
        except:
            print('Error:',time.strftime("%Y/%m/%d-%H:%M:%S", time.localtime()))


In [ ]:
# Endless Loop for API streaming

keywords = ['#donaldtrump', '#joebiden', 'donald trump', 'joe biden']

# crawl between specific time frame (GMT+8 local time)
myStartTime = '0600'
myEndTime = '1000'

while True:
    try:
        # Connect/reconnect the stream
        stream_listener = myStreamListener(mongo_collection, starttime=myStartTime, endtime=myEndTime)
        stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
        stream.filter(track=keywords)
                
    except:
        # Oh well, reconnect and keep tracking
        print('Error (Unknown):',time.strftime("%Y/%m/%d-%H:%M:%S", time.localtime()))
        continue